# DES	simulation	assignment

### 0 Init

In [6]:
import simpy
import numpy as np
import random
import statistics

In [14]:
def DES_queue(size, arrival_rate, service_rate, n):
    env = simpy.Environment()
    servers = simpy.Resource(env, capacity = n)

    inter_arrival_times = []
    service_times = []

    waiting_times = []

    def customer_generator(env, size , servers, arrival_rate, service_rate):
        ''' Generates a costumer after a delay of the arrival time and processes the customer  '''
        for i in range(size):
            dt = random.expovariate(1/arrival_rate)
            inter_arrival_times.append(dt)
            yield env.timeout(dt)
            costomer = customer_processing(env,servers,arrival_rate,service_rate)
            env.process(costomer)

    def customer_processing(env, servers, arrival_rate, service_rate):
        ''' Processed the customer'''
        arrival_time = env.now

        with servers.request() as request:
            yield request  # Wait for a server to become available
            wait_time = env.now - arrival_time
            waiting_times.append(wait_time)  # Log the wait time
            # Service time
            ds = random.expovariate(1/service_rate)
            service_times.append(ds)
            yield env.timeout(ds)

    env.process(customer_generator(env, size, servers,arrival_rate,service_rate))

    env.run()
    
    mean_interarrival = statistics.mean(inter_arrival_times)
    mean_waiting = statistics.mean(waiting_times)
    mean_service = statistics.mean(service_times)

    return [mean_interarrival, mean_waiting, mean_service]


In [16]:
size = 100
system_load = 0.95
mean_service_time = 4
mu = 1/mean_service_time
servers = [1,2,4]
num_simulations = 10

def lamda(n):
    return 1 / (n*system_load * mu)

results = {1: [], 2: [], 4: []}

for n in servers:
    for _ in range(num_simulations):
        result = DES_queue(size, lamda(n), mu, n)
        results[n].append(result[1])

for res in results:
    print(results[res])

[0.021987036583132776, 0.014845075456318604, 0.0003123685704554191, 0.026311455618014533, 0.01714800204954372, 0.0073575483487449845, 0.012282352999174613, 0.014114309375716871, 0.0068253213221670704, 0.007625261342117114]
[0.0010458054804274753, 0.0, 0.0, 0.0, 0.0, 0.003553656258603439, 0.001235375634079503, 0.0, 0.0, 0.0]
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
